In [ ]:
# Set project directory
import os
import sys


def project_dir():
    notebook_path = %pwd
    repo_name = "worldbank_data_exploration"
    repo_folder = notebook_path.split(repo_name)[0]
    return os.path.join(repo_folder, repo_name)


pwd = os.getenv("PWD", project_dir())
os.environ["PWD"] = pwd
sys.path.append(pwd)

import warnings

warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

from data_sources.get import get_indicators
from data_sources.load_dataset import (
    INDICATORS_SELECTED,
    INDICATORS_AGRICULTURE,
    INDICATORS_ECONOMY,
    INDICATORS_HEALTH,
    ALL_INDICATORS,
    INDICATORS_YEARS_RANGE,
)

In [ ]:
df = get_indicators(INDICATORS_HEALTH)

print(df["Indicator Name"].unique())

['Life expectancy at birth, total (years)'
 'Mortality rate, under-5 (per 1,000 live births)'
 'Fertility rate, total (births per woman)'
 'Prevalence of undernourishment (% of population)'
 'Immunization, DPT (% of children ages 12-23 months)'
 'Population growth (annual %)'
 'Age dependency ratio (% of working-age population)'
 'Incidence of tuberculosis (per 100,000 people)'
 'Immunization, measles (% of children ages 12-23 months)'
 'Adolescent fertility rate (births per 1,000 women ages 15-19)'
 'Death rate, crude (per 1,000 people)'
 'Birth rate, crude (per 1,000 people)']


In [ ]:
df = df.pivot_table(
    values="Value", index="Year", columns=["Indicator Name", "Country Name"]
)

In [ ]:
df_nans = df.loc[INDICATORS_YEARS_RANGE].isnull().sum().reset_index()
df_nans

,Indicator Name,Country Name,0
0,"Adolescent fertility rate (births per 1,000 wo...",Afghanistan,0
1,"Adolescent fertility rate (births per 1,000 wo...",Africa Eastern and Southern,0
2,"Adolescent fertility rate (births per 1,000 wo...",Africa Western and Central,0
3,"Adolescent fertility rate (births per 1,000 wo...",Albania,0
4,"Adolescent fertility rate (births per 1,000 wo...",Algeria,0
...,...,...,...
2842,Prevalence of undernourishment (% of population),Vanuatu,1
2843,Prevalence of undernourishment (% of population),"Venezuela, RB",1
2844,Prevalence of undernourishment (% of population),Vietnam,1
2845,Prevalence of undernourishment (% of population),World,1


In [ ]:
aggregates = np.array(
    [
        "Africa Eastern and Southern",
        "Africa Western and Central",
        "Arab World",
        "Caribbean small states",
        "Central Europe and the Baltics",
        "Early-demographic dividend",
        "East Asia & Pacific",
        "East Asia & Pacific (excluding high income)",
        "East Asia & Pacific (IDA & IBRD countries)",
        "Euro area",
        "Europe & Central Asia",
        "Europe & Central Asia (excluding high income)",
        "Europe & Central Asia (IDA & IBRD countries)",
        "European Union",
        "Fragile and conflict affected situations",
        "Heavily indebted poor countries (HIPC)",
        "High income",
        "IBRD only",
        "IDA & IBRD total",
        "IDA blend",
        "IDA only",
        "IDA total",
        "Late-demographic dividend",
        "Latin America & Caribbean",
        "Latin America & Caribbean (excluding high income)",
        "Latin America & the Caribbean (IDA & IBRD countries)",
        "Least developed countries: UN classification",
        "Low & middle income",
        "Low income",
        "Lower middle income",
        "Middle East & North Africa",
        "Middle East & North Africa (excluding high income)",
        "Middle East & North Africa (IDA & IBRD countries)",
        "Middle income",
        "North America",
        "OECD members",
        "Other small states",
        "Pacific island small states",
        "Post-demographic dividend",
        "Pre-demographic dividend",
        "Small states",
        "South Asia",
        "South Asia (IDA & IBRD)",
        "Sub-Saharan Africa",
        "Sub-Saharan Africa (excluding high income)",
        "Sub-Saharan Africa (IDA & IBRD countries)",
        "Upper middle income",
        "World",
    ]
)

aggregates.shape

(48,)

In [ ]:
countries_with_nans = df_nans[df_nans[0] > 2]["Country Name"].unique()
countries_with_nans

array(['Eritrea', 'American Samoa', 'Andorra', 'Curacao', 'Dominica',
       'Isle of Man', 'Marshall Islands', 'Monaco',
       'Northern Mariana Islands', 'San Marino',
       'Sint Maarten (Dutch part)', 'St. Kitts and Nevis', 'Palau',
       'Montenegro', 'South Sudan', 'Serbia', 'Cayman Islands'],
      dtype=object)

In [ ]:
df_cleared = df.loc[
    INDICATORS_YEARS_RANGE.start : INDICATORS_YEARS_RANGE.stop + 1
].stack()
print(df_cleared.shape)
df_cleared.drop(index=countries_with_nans, level=1, inplace=True)
print(df_cleared.shape)
df_cleared.drop(index=aggregates, level=1, inplace=True)
print(df_cleared.shape)

(5300, 12)
(4960, 12)
(4000, 12)


In [ ]:
df_cleared = df_cleared.unstack()
df_cleared.bfill(inplace=True)
df_cleared.ffill(inplace=True)
df_cleared = df_cleared.stack()
df_cleared.dropna(axis=0, inplace=True)

In [ ]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_cleared.values)
df_scaled = pd.DataFrame(
    data=data_scaled, index=df_cleared.index, columns=df_cleared.columns
)
df_scaled.head()

Indicator Name     Adolescent fertility rate (births per 1,000 women ages 15-19)  \
Year Country Name                                                                  
2000 Afghanistan                                            2.306114               
     Albania                                               -0.804539               
     Algeria                                               -0.972428               
     Angola                                                 3.291742               
     Argentina                                              0.286453               

Indicator Name     Age dependency ratio (% of working-age population)  \
Year Country Name                                                       
2000 Afghanistan                                            2.477419    
     Albania                                               -0.091821    
     Algeria                                                0.097692    
     Angola                                                 2.148089    
     Argentina                                              0.025715    

Indicator Name     Birth rate, crude (per 1,000 people)  \
Year Country Name                                         
2000 Afghanistan                               2.415543   
     Albania                                  -0.523233   
     Algeria                                  -0.233123   
     Angola                                    2.427545   
     Argentina                                -0.238241   

Indicator Name     Death rate, crude (per 1,000 people)  \
Year Country Name                                         
2000 Afghanistan                               1.019019   
     Albania                                  -0.761880   
     Algeria                                  -1.054299   
     Angola                                    2.957634   
     Argentina                                -0.169372   

Indicator Name     Fertility rate, total (births per woman)  \
Year Country Name                                             
2000 Afghanistan                                   3.109483   
     Albania                                      -0.503464   
     Algeria                                      -0.261380   
     Angola                                        2.535806   
     Argentina                                    -0.242394   

Indicator Name     Immunization, DPT (% of children ages 12-23 months)  \
Year Country Name                                                        
2000 Afghanistan                                           -4.212668     
     Albania                                                0.685641     
     Algeria                                               -0.052460     
     Angola                                                -3.742967     
     Argentina                                             -0.253761     

Indicator Name     Immunization, measles (% of children ages 12-23 months)  \
Year Country Name                                                            
2000 Afghanistan                                           -3.909625         
     Albania                                                0.607844         
     Algeria                                               -0.388657         
     Angola                                                -3.577458         
     Argentina                                              0.342110         

Indicator Name     Incidence of tuberculosis (per 100,000 people)  \
Year Country Name                                                   
2000 Afghanistan                                         0.212351   
     Albania                                            -0.586791   
     Algeria                                            -0.334681   
     Angola                                              0.721328   
     Argentina                                          -0.515439   

Indicator Name     Life expectancy at birth, total (years)  \
Year Country Name

In [ ]:
from data_sources.load_dataset import load_time_series, load_dataset

df = load_dataset(indicators=INDICATORS_HEALTH)
print(df.isnull().sum().sum())
df

0


Indicator Name      Adolescent fertility rate (births per 1,000 women ages 15-19)  \
Year Country Name                                                                   
2000 Afghanistan                                             153.9510               
     Albania                                                  19.0372               
     Algeria                                                  11.7556               
     Angola                                                  196.6992               
     Argentina                                                66.3552               
...                                                               ...               
2018 Uzbekistan                                               24.2094               
     Vanuatu                                                  48.9082               
     Venezuela, RB                                            84.9792               
     Vietnam                                                  29.1518               
     Yemen, Rep.                                              59.1624               

Indicator Name      Age dependency ratio (% of working-age population)  \
Year Country Name                                                        
2000 Afghanistan                                           104.856398    
     Albania                                                59.723586    
     Algeria                                                63.052679    
     Angola                                                 99.071186    
     Argentina                                              61.788296    
...                                                               ...    
2018 Uzbekistan                                             49.488438    
     Vanuatu                                                73.878043    
     Venezuela, RB                                          53.689245    
     Vietnam                                                43.776605    
     Yemen, Rep.                                            73.886310    

Indicator Name      Birth rate, crude (per 1,000 people)  \
Year Country Name                                          
2000 Afghanistan                                  48.021   
     Albania                                      16.436   
     Algeria                                      19.554   
     Angola                                       48.150   
     Argentina                                    19.499   
...                                                  ...   
2018 Uzbekistan                                   23.300   
     Vanuatu                                      29.595   
     Venezuela, RB                                17.881   
     Vietnam                                      16.745   
     Yemen, Rep.                                  30.453   

Indicator Name      Death rate, crude (per 1,000 people)  \
Year Country Name                                          
2000 Afghanistan                                  11.718   
     Albania                                       5.914   
     Algeria                                       4.961   
     Angola                                       18.036   
     Argentina                                     7.845   
...                                                  ...   
2018 Uzbekistan                                    4.700   
     Vanuatu                                       5.266   
     Venezuela, RB                                 6.982   
     Vietnam                                       6.317   
     Yemen, Rep.                                   5.983   

Indicator Name      Fertility rate, total (births per woman)  \
Year Country Name                                              
2000 Afghanistan                                       7.485   
     Albania                                           2.157   
     Algeria                                           2.514   
     Angola                                      

---

In [ ]:
all_time_series = load_time_series(indicators=INDICATORS_SELECTED)
all_time_series[1], len(all_time_series[1])

(array(['Albania', 'Angola', 'Australia', 'Austria', 'Azerbaijan',
        'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Brazil',
        'Bulgaria', 'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada',
        'Chad', 'Chile', 'China', 'Colombia', 'Congo, Dem. Rep.',
        'Costa Rica', 'Croatia', 'Czech Republic', 'Denmark',
        'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador',
        'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Ghana',
        'Greece', 'Guatemala', 'Honduras', 'Hungary', 'India', 'Indonesia',
        'Iran, Islamic Rep.', 'Ireland', 'Israel', 'Italy', 'Jamaica',
        'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Korea, Rep.', 'Kuwait',
        'Kyrgyz Republic', 'Latvia', 'Lithuania', 'Madagascar', 'Mali',
        'Mexico', 'Moldova', 'Mongolia', 'Morocco', 'Nepal', 'Netherlands',
        'New Zealand', 'Nicaragua', 'Nigeria', 'North Macedonia', 'Norway',
        'Oman', 'Pakistan', 'Panama', 'Paraguay', 'Peru', 'Philippines',
    

In [ ]:
all_time_series = load_time_series(indicators=INDICATORS_AGRICULTURE)
all_time_series[1], len(all_time_series[1])

(array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
        'Australia', 'Austria', 'Azerbaijan', 'Bangladesh', 'Belarus',
        'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia',
        'Bosnia and Herzegovina', 'Brazil', 'Brunei Darussalam',
        'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
        'Canada', 'Chile', 'China', 'Colombia', 'Congo, Dem. Rep.',
        'Congo, Rep.', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba',
        'Cyprus', 'Czech Republic', 'Denmark', 'Dominican Republic',
        'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Estonia',
        'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia, The',
        'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea',
        'Guyana', 'Honduras', 'Hungary', 'India', 'Indonesia',
        'Iran, Islamic Rep.', 'Iraq', 'Ireland', 'Israel', 'Italy',
        'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Korea, Rep.',
        'Kuwait', 'Kyrgyz Republic', 'L

In [ ]:
all_time_series = load_time_series(indicators=INDICATORS_ECONOMY)
all_time_series[1], len(all_time_series[1])

(array(['Albania', 'Angola', 'Australia', 'Azerbaijan', 'Bahamas, The',
        'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Botswana',
        'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burundi', 'Cambodia',
        'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Congo, Rep.',
        'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark',
        'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Estonia',
        'Eswatini', 'Fiji', 'Finland', 'France', 'Georgia', 'Germany',
        'Ghana', 'Guatemala', 'Haiti', 'Honduras', 'Hong Kong SAR, China',
        'Hungary', 'Iceland', 'India', 'Indonesia', 'Israel', 'Italy',
        'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Korea, Rep.',
        'Kyrgyz Republic', 'Lao PDR', 'Latvia', 'Lithuania', 'Luxembourg',
        'Malaysia', 'Malta', 'Mauritius', 'Mexico', 'Moldova', 'Mongolia',
        'Morocco', 'Nepal', 'Netherlands', 'New Zealand', 'Niger',
        'Nigeria', 'North Macedonia', 'Norway', 'Oman', 'Pakistan

In [ ]:
all_time_series = load_time_series(indicators=INDICATORS_HEALTH)
all_time_series[1], len(all_time_series[1])

(array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
        'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bangladesh',
        'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bolivia',
        'Bosnia and Herzegovina', 'Botswana', 'Brazil',
        'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Cabo Verde',
        'Cambodia', 'Cameroon', 'Canada', 'Central African Republic',
        'Chad', 'Chile', 'China', 'Colombia', 'Congo, Dem. Rep.',
        'Congo, Rep.', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba',
        'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti',
        'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador',
        'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
        'Gabon', 'Gambia, The', 'Georgia', 'Germany', 'Ghana', 'Greece',
        'Guatemala', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland',
        'India', 'Indonesia', 'Iran, Islamic Rep.', 'Iraq', 'Ireland',
        'Israel', 'Italy', '

In [ ]:
all_time_series = load_time_series(indicators=ALL_INDICATORS)
all_time_series[1], len(all_time_series[1])

(array(['Albania', 'Angola', 'Australia', 'Azerbaijan', 'Bangladesh',
        'Belarus', 'Belgium', 'Bolivia', 'Brazil', 'Bulgaria', 'Cambodia',
        'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica',
        'Croatia', 'Czech Republic', 'Denmark', 'Ecuador',
        'Egypt, Arab Rep.', 'El Salvador', 'Estonia', 'Finland', 'France',
        'Georgia', 'Germany', 'Ghana', 'Guatemala', 'Honduras', 'Hungary',
        'India', 'Indonesia', 'Israel', 'Italy', 'Jamaica', 'Japan',
        'Jordan', 'Kazakhstan', 'Kenya', 'Korea, Rep.', 'Kyrgyz Republic',
        'Latvia', 'Lithuania', 'Mexico', 'Mongolia', 'Morocco', 'Nepal',
        'Netherlands', 'New Zealand', 'Nigeria', 'North Macedonia',
        'Norway', 'Oman', 'Pakistan', 'Panama', 'Paraguay', 'Peru',
        'Philippines', 'Poland', 'Portugal', 'Romania',
        'Russian Federation', 'Saudi Arabia', 'Senegal', 'Slovak Republic',
        'South Africa', 'Spain', 'Sri Lanka', 'Sweden', 'Switzerland',
        'Tanzani